# Job Listings Data Cleaning

This notebook takes a dataset of joblistings and cleans it into a more usable format.

 - Duplicates are removed, keywords from the job title are extracted
 - The salary estimate range is split into min and max columns with numerical type
 - Anomalies are removed
 - The size and revenue ranges are converted approximante values in numerical type
 - Other null or problematic values are dealt with

In [302]:
import pandas as pd
import numpy as np
import re

In [304]:
# import the dataset and look at the first few rows

df = pd.read_csv(r"C:/Users/mattc/OneDrive/Documents/Data/Python/DS Job Listings.csv")
df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


## Look for and remove any duplicates 

In [346]:
duplicates = df[df.duplicated(keep=False)]
duplicates

,Job Title,level_keywords,role_keywords,Salary Estimate,min_salary_est($K),max_salary_est($K),Job Description,Rating,Company Name,Location,Headquarters,Size,approx_employees,Founded,Type of ownership,Industry,Sector,Revenue,approx_revenue($Mil),Competitors
131,Senior Data Engineer,senior,"data, engineer",$90K-$109K (Glassdoor est.),90,109,Lendio is looking to fill a position for a Sen...,4.9,Lendio\n4.9,"Lehi, UT","Lehi, UT",201 to 500 employees,350,2011,Company - Private,Lending,Finance,$50 to $100 million (USD),75.0,
134,Machine Learning Engineer,None,"machine learning, engineer",$90K-$109K (Glassdoor est.),90,109,Role Description\nTriplebyte screens and evalu...,3.2,Triplebyte\n3.2,Remote,"San Francisco, CA",51 to 200 employees,125,2015,Company - Private,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,<NA>,
135,Machine Learning Engineer,None,"machine learning, engineer",$90K-$109K (Glassdoor est.),90,109,Role Description\nTriplebyte screens and evalu...,3.2,Triplebyte\n3.2,Remote,"San Francisco, CA",51 to 200 employees,125,2015,Company - Private,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,<NA>,
136,Senior Data Engineer,senior,"data, engineer",$90K-$109K (Glassdoor est.),90,109,Lendio is looking to fill a position for a Sen...,4.9,Lendio\n4.9,"Lehi, UT","Lehi, UT",201 to 500 employees,350,2011,Company - Private,Lending,Finance,$50 to $100 million (USD),75.0,
357,Data Scientist,None,"data, scientist",$122K-$146K (Glassdoor est.),122,146,Job Overview: The Data Scientist is a key memb...,NaN,Hatch Data Inc,"San Francisco, CA",,,<NA>,-1,,,,,<NA>,
358,Data Scientist,None,"data, scientist",$122K-$146K (Glassdoor est.),122,146,Job Overview: The Data Scientist is a key memb...,NaN,Hatch Data Inc,"San Francisco, CA",,,<NA>,-1,,,,,<NA>,
359,Data Scientist,None,"data, scientist",$122K-$146K (Glassdoor est.),122,146,Job Overview: The Data Scientist is a key memb...,NaN,Hatch Data Inc,"San Francisco, CA",,,<NA>,-1,,,,,<NA>,
360,Data Scientist,None,"data, scientist",$122K-$146K (Glassdoor est.),122,146,Job Overview: The Data Scientist is a key memb...,NaN,Hatch Data Inc,"San Francisco, CA",,,<NA>,-1,,,,,<NA>,
361,Data Scientist,None,"data, scientist",$122K-$146K (Glassdoor est.),122,146,Job Overview: The Data Scientist is a key memb...,NaN,Hatch Data Inc,"San Francisco, CA",,,<NA>,-1,,,,,<NA>,
362,Data Scientist,None,"data, scientist",$122K-$146K (Glassdoor est.),122,146,Job Overview: The Data Scientist is a key memb...,NaN,Hatch Data Inc,"San Francisco, CA",,,<NA>,-1,,,,,<NA>,


In [352]:
df = df.drop_duplicates()

In [356]:
df.duplicated().sum()

0

## Identifying keywords in the job titles

In [310]:
len(df['Job Title'].unique())              #number of different job titles in the table

172

In [152]:
# displays all unique job titles to scan for common keywords

df['Job Title'].unique()

array(['Sr Data Scientist', 'Data Scientist',
       'Data Scientist / Machine Learning Expert',
       'Staff Data Scientist - Analytics',
       'Data Scientist - Statistics, Early Career', 'Data Modeler',
       'Experienced Data Scientist', 'Data Scientist - Contract',
       'Data Analyst II', 'Medical Lab Scientist',
       'Data Scientist/Machine Learning', 'Human Factors Scientist',
       'Business Intelligence Analyst I- Data Insights',
       'Data Scientist - Risk', 'Data Scientist-Human Resources',
       'Senior Research Statistician- Data Scientist', 'Data Engineer',
       'Associate Data Scientist', 'Business Intelligence Analyst',
       'Senior Analyst/Data Scientist', 'Data Analyst',
       'Machine Learning Engineer', 'Data Analyst I',
       'Scientist - Molecular Biology',
       'Computational Scientist, Machine Learning',
       'Senior Data Scientist', 'Jr. Data Engineer',
       'E-Commerce Data Analyst', 'Data Analytics Engineer',
       'Product Data Scient

In [312]:
# list of keywords relating to the level of the job

level_keywords = ['sr', 'senior', 'jr', 'junior', 'experienced', 'staff', 'principal', 'lead', 'manager', 'chief', 'graduate', 'vice', 'president', 'expert']

# list of keywords relating to the role of the job

role_keywords = ['software', 'data', 'business', 'computer', 'computational', 'risk', 'scientist', 'science', 'machine learning', 'ml', 'ai', 'analyst', 'analytics', 'engineer', 'engineering', 'programmer', 'programming', 'developer']

In [314]:
# finds any of the job level keywords in the job title

def extract_level_keywords(title):
    found_keywords = [kw for kw in level_keywords if re.search(rf'\b{kw}\b', title, re.IGNORECASE)]
    return ', '.join(found_keywords) if found_keywords else None

In [316]:
# finds any of the job role keywords in the job title

def extract_role_keywords(title):
    found_keywords = [kw for kw in role_keywords if re.search(rf'\b{kw}\b', title, re.IGNORECASE)]
    return ', '.join(found_keywords) if found_keywords else None

In [318]:
# applies the above functions to the job title column and creates new columns with the keywords in

df['level_keywords'] = df['Job Title'].apply(extract_level_keywords)
df['role_keywords'] = df['Job Title'].apply(extract_role_keywords)

#reorders columns to put keywords next to job title

df = df[['Job Title', 'level_keywords', 'role_keywords', 'Salary Estimate', 'Job Description', 'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors']]
df.head()

,Job Title,level_keywords,role_keywords,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Sr Data Scientist,sr,"data, scientist",$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


## Separating salary estimate range into min and max

In [360]:
# taking the first number in the salary estimate column

min = df['Salary Estimate'].str.split('-').str[0]
df['min_salary_est($K)'] = min.str.replace(r'[^\d]', '', regex=True).astype(int)

# taking the second number in the salary estimate column

max = df['Salary Estimate'].str.split('-').str[1]
df['max_salary_est($K)'] = max.str.replace(r'[^\d]', '', regex=True).astype(int)

# reordering the columns to put the min and max salary estimate next to the salary estimate range

df = df[['Job Title', 'level_keywords', 'role_keywords', 'Salary Estimate', 'min_salary_est($K)', 'max_salary_est($K)', 'Job Description', 'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors']]
df.head()

,Job Title,level_keywords,role_keywords,Salary Estimate,min_salary_est($K),max_salary_est($K),Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Sr Data Scientist,sr,"data, scientist",$137K-$171K (Glassdoor est.),137,171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),137,171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),
2,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),137,171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),
3,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),137,171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),137,171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


## Checking the ratings for anomalies

In [39]:
df['Rating'].max()

5.0

In [43]:
df['Rating'].min()

-1.0

In [71]:
# the ratings should not be negative so now searching for any negative ratings

for i in range(len(df['Rating'])):
    if df['Rating'][i] < 0:
        print(df.index[i], df['Rating'][i])

154 -1.0
158 -1.0
230 -1.0
282 -1.0
285 -1.0
290 -1.0
319 -1.0
322 -1.0
329 -1.0
338 -1.0
351 -1.0
357 -1.0
358 -1.0
359 -1.0
360 -1.0
361 -1.0
362 -1.0
388 -1.0
389 -1.0
409 -1.0
411 -1.0
425 -1.0
430 -1.0
431 -1.0
437 -1.0
438 -1.0
440 -1.0
457 -1.0
459 -1.0
495 -1.0
496 -1.0
497 -1.0
498 -1.0
499 -1.0
500 -1.0
504 -1.0
519 -1.0
524 -1.0
555 -1.0
568 -1.0
613 -1.0
615 -1.0
637 -1.0
650 -1.0
656 -1.0
657 -1.0
660 -1.0
664 -1.0
668 -1.0
669 -1.0


In [328]:
# there are a number of ratings at -1.0, these full rows are all displayed below

negatives = []

for i in range(len(df['Rating'])):
    if df['Rating'][i] < 0:
        negatives.append(df.index[i])

df.loc[negatives]

,Job Title,level_keywords,role_keywords,Salary Estimate,min_salary_est($K),max_salary_est($K),Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
154,ELISA RESEARCH SCIENTIST (CV-15),None,scientist,$90K-$109K (Glassdoor est.),90,109,"Covaxx, a subsidiary of the UBI Group, has joi...",-1.0,Covid-19 Search Partners,"Hauppauge, NY",-1,-1,-1,-1,-1,-1,-1,-1
158,Machine Learning Engineer,None,"machine learning, engineer",$101K-$165K (Glassdoor est.),101,165,Overview\n\nRadical Convergence is a fast-pace...,-1.0,Radical Convergence,"Reston, VA",-1,-1,-1,-1,-1,-1,-1,-1
230,Data Scientist,None,"data, scientist",$71K-$123K (Glassdoor est.),71,123,Senior Data Scientist Active Secret clearance ...,-1.0,"Encode, Inc.","Norfolk, VA","Manalapan, NJ",1 to 50 employees,-1,Company - Private,IT Services,Information Technology,$5 to $10 million (USD),-1
282,Data Scientist,None,"data, scientist",$91K-$150K (Glassdoor est.),91,150,Greetings from Trovetechs!!!\n\nÂ\n\nWe have a...,-1.0,Trovetechs Inc,"San Francisco, CA","Hillsborough, NJ",Unknown,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1
285,Data Scientist,None,"data, scientist",$141K-$225K (Glassdoor est.),141,225,Greetings from Trovetechs!!!\n\nÂ\n\nWe have a...,-1.0,Trovetechs Inc,"San Francisco, CA","Hillsborough, NJ",Unknown,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1
290,Data Scientist,None,"data, scientist",$141K-$225K (Glassdoor est.),141,225,Work Authorization Those authorized to work in...,-1.0,PETADATA,"San Francisco, CA","Fremont, CA",51 to 200 employees,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1
319,Data Scientist,None,"data, scientist",$145K-$225K(Employer est.),145,225,"Job Description\nSelecting features, building ...",-1.0,Microagility,"New York, NY","Princeton, NJ",1 to 50 employees,-1,Company - Private,Consulting,Business Services,$1 to $5 million (USD),-1
322,Data Scientist,None,"data, scientist",$145K-$225K(Employer est.),145,225,Job Description\nData Scientist\nAre you inter...,-1.0,"Rainmaker Resources, LLC","Cincinnati, OH","Cincinnati, OH",1 to 50 employees,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1
329,Data Scientist,None,"data, scientist",$79K-$147K (Glassdoor est.),79,147,Description:\n\nPosition Requires a Top Secret...,-1.0,B4Corp,"Reston, VA","Leesburg, VA",51 to 200 employees,-1,Company - Private,IT Services,Information Technology,Unknown / Non-Applicable,-1
338,Data Scientist,None,"data, scientist",$79K-$147K (Glassdoor est.),79,147,Work in a fast growing startup with unlimited ...,-1.0,Stride Search,"San Francisco, CA","Westlake Village, CA",1 to 50 employees,-1,Company - Private,-1,-1,Less than $1 million (USD),-1


In [362]:
# this reveals that -1 seems to be the default value for missing data in the rating column as well as others

In [330]:
# for the ratings, these -1.0 values are changed to empty cells so that their values don't affect calculations such as averages

df['Rating'] = df['Rating'].replace(-1.0, np.nan)

In [332]:
# for the other columns, the -1 values are replaced with blank cells 

df = df.replace('-1', '')

## Turning the size column into numerical

In [364]:
# showing all the different values in the size column to see if there are any non-standard ones

df['Size'].unique()

array(['1001 to 5000 employees', '5001 to 10000 employees',
       '501 to 1000 employees', '51 to 200 employees', '10000+ employees',
       '201 to 500 employees', '1 to 50 employees', '', 'Unknown'],
      dtype=object)

In [334]:
# creating a new column with the approximate number of employees being in the middle of the range

df['approx_employees'] = df['Size'].replace({
                                '1 to 50 employees':25, 
                                '51 to 200 employees':125,
                                '201 to 500 employees':350,
                                '501 to 1000 employees':750,
                                '1001 to 5000 employees':2500,
                                '5001 to 10000 employees':7500,
                                '10000+ employees':15000,
                                'Unknown':pd.NA,
                                '':pd.NA})

df['approx_employees'] = df['approx_employees'].astype('Int64')

#reordering columns to put size and approx employees together

df = df[['Job Title', 'level_keywords', 'role_keywords', 'Salary Estimate', 'min_salary_est($K)', 'max_salary_est($K)', 'Job Description', 'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'approx_employees', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors']]
df.head()

,Job Title,level_keywords,role_keywords,Salary Estimate,min_salary_est($K),max_salary_est($K),Job Description,Rating,Company Name,Location,Headquarters,Size,approx_employees,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Sr Data Scientist,sr,"data, scientist",$137K-$171K (Glassdoor est.),137,171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,2500,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),137,171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,7500,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),
2,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),137,171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,2500,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),
3,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),137,171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,750,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),137,171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,125,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


## Looking for issues with the locations

In [366]:
# displaying the unique locations to scan for any unusual entries

sorted(df['Location'].unique())

['Adelphi, MD',
 'Akron, OH',
 'Alexandria, VA',
 'Alpharetta, GA',
 'Ann Arbor, MI',
 'Annapolis Junction, MD',
 'Appleton, WI',
 'Arlington, VA',
 'Ashburn, VA',
 'Atlanta, GA',
 'Aurora, CO',
 'Austin, TX',
 'Baltimore, MD',
 'Beavercreek, OH',
 'Bedford, MA',
 'Bellevue, WA',
 'Bethesda, MD',
 'Birmingham, AL',
 'Blue Bell, PA',
 'Boston, MA',
 'Bothell, WA',
 'Boulder, CO',
 'Bridgeport, WV',
 'Brisbane, CA',
 'Brooklyn, NY',
 'Burbank, CA',
 'Burlingame, CA',
 'California',
 'Cambridge, MA',
 'Carmel, IN',
 'Carpinteria, CA',
 'Carson, CA',
 'Cedar Rapids, IA',
 'Champaign, IL',
 'Chandler, AZ',
 'Chantilly, VA',
 'Charlotte, NC',
 'Chicago, IL',
 'Cincinnati, OH',
 'Cleveland, OH',
 'Collegeville, PA',
 'Colorado Springs, CO',
 'Columbia, MD',
 'Columbia, MO',
 'Columbia, SC',
 'Concord, CA',
 'Coral Gables, FL',
 'Culver City, CA',
 'Cupertino, CA',
 'Dallas, TX',
 'Danvers, MA',
 'Dayton, OH',
 'Dearborn, MI',
 'Denver, CO',
 'Durham, NC',
 'Edison, NJ',
 'Emeryville, CA',
 'E

In [165]:
# no duplicates, all have same format except 'Utah' and 'United States' but insufficient data to improve these

## Looking for issues with the types of ownership

In [368]:
# displaying the unique types of ownership to scan for any unusual entries

df['Type of ownership'].unique()

array(['Nonprofit Organization', 'Company - Public',
       'Private Practice / Firm', 'Company - Private', 'Government',
       'Subsidiary or Business Segment', 'Other Organization', '',
       'Hospital', 'Self-employed', 'College / University', 'Contract'],
      dtype=object)

In [336]:
# replacing unknown with empty cells, otherwise no issues

df['Type of ownership'] = df['Type of ownership'].replace('Unknown', '')

## Looking at issues with industry and sector

In [370]:
# displaying the unique industries to scan for any unusual entries

sorted(df['Industry'].unique())

['',
 'Accounting',
 'Advertising & Marketing',
 'Aerospace & Defense',
 'Architectural & Engineering Services',
 'Banks & Credit Unions',
 'Biotech & Pharmaceuticals',
 'Cable, Internet & Telephone Providers',
 'Chemical Manufacturing',
 'Colleges & Universities',
 'Computer Hardware & Software',
 'Construction',
 'Consulting',
 'Consumer Electronics & Appliances Stores',
 'Consumer Products Manufacturing',
 'Department, Clothing, & Shoe Stores',
 'Electrical & Electronic Manufacturing',
 'Energy',
 'Enterprise Software & Network Solutions',
 'Express Delivery Services',
 'Farm Support Services',
 'Federal Agencies',
 'Financial Transaction Processing',
 'Food & Beverage Manufacturing',
 'Food & Beverage Stores',
 'Health Care Services & Hospitals',
 'Health, Beauty, & Fitness',
 'Hotels, Motels, & Resorts',
 'IT Services',
 'Industrial Manufacturing',
 'Insurance Agencies & Brokerages',
 'Insurance Carriers',
 'Internet',
 'Investment Banking & Asset Management',
 'Lending',
 'Logist

In [372]:
# displaying the unique sectors to scan for any unusual entries

sorted(df['Sector'].unique())

['',
 'Accounting & Legal',
 'Aerospace & Defense',
 'Agriculture & Forestry',
 'Biotech & Pharmaceuticals',
 'Business Services',
 'Construction, Repair & Maintenance',
 'Consumer Services',
 'Education',
 'Finance',
 'Government',
 'Health Care',
 'Information Technology',
 'Insurance',
 'Manufacturing',
 'Media',
 'Non-Profit',
 'Oil, Gas, Energy & Utilities',
 'Real Estate',
 'Retail',
 'Telecommunications',
 'Transportation & Logistics',
 'Travel & Tourism']

In [190]:
# no issues

## Converting the revenue to numerical

In [374]:
# displaying the unique revenue ranges to see if there are any non-standard ones

df['Revenue'].unique()

array(['Unknown / Non-Applicable', '$1 to $2 billion (USD)',
       '$100 to $500 million (USD)', '$10+ billion (USD)',
       '$2 to $5 billion (USD)', '$500 million to $1 billion (USD)',
       '$5 to $10 billion (USD)', '$10 to $25 million (USD)',
       '$25 to $50 million (USD)', '$50 to $100 million (USD)',
       '$1 to $5 million (USD)', '$5 to $10 million (USD)',
       'Less than $1 million (USD)', ''], dtype=object)

In [344]:
# creating a new column with the approximate revenue being in the middle of the range, replacing empty or unknown values with NaN

df['approx_revenue($Mil)'] = df['Revenue'].replace({
                                'Less than $1 million (USD)':0.5, 
                                '$1 to $5 million (USD)':2.5,
                                '$5 to $10 million (USD)':7.5,
                                '$10 to $25 million (USD)':17.5,
                                '$25 to $50 million (USD)':37.5,
                                '$50 to $100 million (USD)':75,
                                '$100 to $500 million (USD)':300,
                                '$500 million to $1 billion (USD)':750,
                                '$1 to $2 billion (USD)':1500,
                                '$2 to $5 billion (USD)':3750,
                                '$5 to $10 billion (USD)':7500,
                                '$10+ billion (USD)':15000,
                                'Unknown / Non-Applicable':pd.NA,
                                '':pd.NA})

df['approx_revenue($Mil)'] = df['approx_revenue($Mil)'].astype('Float64')

# reordeing the columns to put revenue and approx revenue together

df = df[['Job Title', 'level_keywords', 'role_keywords', 'Salary Estimate', 'min_salary_est($K)', 'max_salary_est($K)', 'Job Description', 'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'approx_employees', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'approx_revenue($Mil)', 'Competitors']]
df.head()

,Job Title,level_keywords,role_keywords,Salary Estimate,min_salary_est($K),max_salary_est($K),Job Description,Rating,Company Name,Location,Headquarters,Size,approx_employees,Founded,Type of ownership,Industry,Sector,Revenue,approx_revenue($Mil),Competitors
0,Sr Data Scientist,sr,"data, scientist",$137K-$171K (Glassdoor est.),137,171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,2500,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,<NA>,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),137,171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,7500,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),1500.0,
2,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),137,171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,2500,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),300.0,
3,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),137,171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,750,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),300.0,"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,None,"data, scientist",$137K-$171K (Glassdoor est.),137,171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,125,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,<NA>,"Commerce Signals, Cardlytics, Yodlee"


## Looking for null values

In [232]:
df.isna().sum()

Job Title                 0
level_keywords          557
role_keywords             0
Salary Estimate           0
min_salary_est($K)        0
max_salary_est($K)        0
Job Description           0
Rating                   50
Company Name              0
Location                  0
Headquarters              0
Size                      0
approx_employees         17
Founded                   0
Type of ownership         0
Industry                  0
Sector                    0
Revenue                   0
approx_revenue($Mil)    240
Competitors               0
dtype: int64

In [222]:
# NaN values only appear in the created columns and the rating column which isn't an issue

In [234]:
(df=='').sum()

Job Title                 0
level_keywords            0
role_keywords             0
Salary Estimate           0
min_salary_est($K)        0
max_salary_est($K)        0
Job Description           0
Rating                    0
Company Name              0
Location                  0
Headquarters             31
Size                     27
approx_employees         27
Founded                 118
Type of ownership        31
Industry                 71
Sector                   71
Revenue                  27
approx_revenue($Mil)      0
Competitors             501
dtype: int64

In [376]:
# several columns with blank values but insufficient data to populate them

## Dataset is now clean and ready to be used for analysis